In [ ]:
pip install leafmap

In [ ]:
import geopandas as gpd
import leafmap
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

def load_vector_data(filepath):
    """Reads vector data into a GeoDataFrame."""
    gdf = gpd.read_file(filepath)
    return gdf

def visualize_polygon_data(gdf, map_object):
    """Visualizes polygon data on the map with popups."""
    geojson = gdf.to_json()
    map_object.add_geojson(
        geojson,
        layer_name="Buildings Data",
        style={
            "fillOpacity": 0.4,
            "color": "blue",
            "weight": 1.5
        }
    )
    return "Buildings Data"

def visualize_point_data(gdf, map_object):
    """Visualizes point data on the map with popups."""
    df = pd.DataFrame({
        'lon': gdf.geometry.x,
        'lat': gdf.geometry.y,
        'osm_id': gdf['osm_id'],
        'fclass': gdf['fclass'],
        'name': gdf['name']
    })

    df['popup'] = df.apply(lambda row: f"""
        <strong>Feature Details:</strong><br>
        <strong>ID:</strong> {row['osm_id']}<br>
        <strong>Class:</strong> {row['fclass']}<br>
        <strong>Name:</strong> {row['name']}<br>
        """, axis=1)

    map_object.add_points_from_xy(
        df,
        x='lon',
        y='lat',
        popup='popup',
        layer_name="Places Data",
        icon_names=["info-sign"],
        icon_colors=["blue"]
    )
    return "Places Data"

def visualize_road_data(gdf, map_object):
    """Visualizes road data on the map with popups and classification by road type."""
    color_map = {
        'primary': 'yellow',    # Primary roads
        'secondary': 'orange',  # Secondary roads
        'tertiary': 'green',    # Tertiary roads
        'residential': 'blue',   # Residential roads
        'unclassified': 'gray'   # Unclassified roads
    }

    gdf['color'] = gdf['fclass'].map(color_map).fillna('black')  # Default to black if not found

    geojson = gdf.to_json()
    map_object.add_geojson(
        geojson,
        layer_name="Roads Data",
        style_callback=lambda feature: {
            "color": feature['properties']['color'],  # Access the color from properties
            "weight": 2  # Adjust line weight for visibility
        },
        popup=['osm_id', 'fclass']
    )
    return "Roads Data"

# File paths for polygon, point, and road shapefiles
polygon_shapefile = '/content/gis_osm_buildings_city_gilgit.shp'  # Replace with the path to your polygon shapefile
point_shapefile = '/content/gis_osm_places_city_gilgit.shp'  # Replace with the path to your point shapefile
road_shapefile = '/content/gis_osm_roads_city_gilgit.shp'  # Replace with the path to your road shapefile

# Load polygon, point, and road data
polygon_gdf = load_vector_data(polygon_shapefile)
point_gdf = load_vector_data(point_shapefile)
road_gdf = load_vector_data(road_shapefile)

# Validate geometries
polygon_gdf = polygon_gdf[polygon_gdf.is_valid]
point_gdf = point_gdf[point_gdf.is_valid]
road_gdf = road_gdf[road_gdf.is_valid]

# Initialize the LeafMap map centered on the average location of all datasets
center_lat = (polygon_gdf.geometry.centroid.y.mean() + point_gdf.geometry.centroid.y.mean() + road_gdf.geometry.centroid.y.mean()) / 3
center_lon = (polygon_gdf.geometry.centroid.x.mean() + point_gdf.geometry.centroid.x.mean() + road_gdf.geometry.centroid.x.mean()) / 3
m = leafmap.Map(center=(center_lat, center_lon), zoom=12)
m.add_basemap("SATELLITE")  # Google Satellite as basemap

# Visualize polygon, point, and road data
building_layer = visualize_polygon_data(polygon_gdf, m)
places_layer = visualize_point_data(point_gdf, m)
road_layer = visualize_road_data(road_gdf, m)

# Create radio buttons for layer control
layer_selector = widgets.RadioButtons(
    options=["All", "Buildings Data", "Places Data", "Roads Data"],
    value="All",  # Default to show all layers
    description='Visible Layers:',
    disabled=False
)

def update_layer_visibility(change):
    """Update the visibility of the layers based on radio button selection."""
    selected_layer = change['new']

    # Remove all layers first
    m.clear_layers()

    # Add selected layer(s)
    if selected_layer == "All":
        m.add_geojson(polygon_gdf.to_json(), layer_name="Buildings Data", style={"fillOpacity": 0.4, "color": "blue", "weight": 1.5})
        visualize_point_data(point_gdf, m)
        visualize_road_data(road_gdf, m)
    elif selected_layer == "Buildings Data":
        visualize_polygon_data(polygon_gdf, m)
    elif selected_layer == "Places Data":
        visualize_point_data(point_gdf, m)
    elif selected_layer == "Roads Data":
        visualize_road_data(road_gdf, m)

# Observe changes to the layer selector
layer_selector.observe(update_layer_visibility, names='value')

# Display the layer control
display(layer_selector)

# Display the map
m
